<a href="https://colab.research.google.com/github/aaronjoseph/NLP_Boiler_Plate/blob/main/Data_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install mlflow
#Load the libraries
from mlflow import log_metric, log_param, log_artifacts
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import os
from tqdm import tqdm
import collections, numpy
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
tqdm.pandas()
from sklearn.model_selection import train_test_split,KFold
nltk.download('stopwords')
! pip install neptune-client==0.4.132
import neptune
neptune.init(
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiOGI1M2QwMmItZGQyMi00MTBmLThhZTktZDY0OThjYmZlMGMyIn0=",
    project_qualified_name="aaronjosephmathew/sandbox"
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Project(aaronjosephmathew/sandbox)

In [ ]:
sentiment_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/training.1600000.processed.noemoticon.csv',header=None,error_bad_lines=False,engine='python')
sentiment_data = sentiment_data.sample(n=1_00_000,replace=True)

OSError: ignored

In [ ]:
# Data Cleaning Process
def preprocess(text):
  text=text.lower()
  # remove hyperlinks
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
  text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
  #remove hashtag sign
  text=re.sub(r"#","",text)   
  #remove mentions
  text = re.sub(r"(?:\@)\w+", '', text)
  #remove non ascii chars
  text=text.encode("ascii",errors="ignore").decode()
  #remove some puncts (except . ! ?)
  text=re.sub(r'[:"#$%&\*+,-/:;<=>@\\^_`{|}~]+','',text)
  text=re.sub(r'[!]+','!',text)
  text=re.sub(r'[?]+','?',text)
  text=re.sub(r'[.]+','.',text)
  text=re.sub(r"'","",text)
  text=re.sub(r"\(","",text)
  text=re.sub(r"\)","",text)    
  text=" ".join(text.split())
  return text

def simple_stemmer(text):
  ps=nltk.porter.PorterStemmer()
  text= ' '.join([ps.stem(word) for word in text.split()])
  return text

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
  #set stopwords to english
  stop=set(stopwords.words('english'))
  # Tokenization
  tokenizer=ToktokTokenizer()
  #Setting English stopwords
  stopword_list=nltk.corpus.stopwords.words('english')
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  if is_lower_case:
      filtered_tokens = [token for token in tokens if token not in stopword_list]
  else:
      filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)    
  return filtered_text

In [ ]:
#Apply function
sentiment_data['clean']=sentiment_data[5].progress_apply(preprocess)
sentiment_data['stem']=sentiment_data['clean'].progress_apply(simple_stemmer)
sentiment_data['token'] = sentiment_data['stem'].progress_apply(remove_stopwords) 

100%|██████████| 1600000/1600000 [08:40<00:00, 3076.54it/s]


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, auc, roc_curve, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
sentiment_data['output'] = le.fit_transform(sentiment_data[0])
# Splitting the data - Into train_test
X = sentiment_data['token']
y = sentiment_data['output']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42) 

In [ ]:
# Count Vectorizer for Bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(X_train)
#transformed test reviews
cv_test_reviews=cv.transform(X_test)
# TFIDF
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(X_train)
#transformed test reviews
tv_test_reviews=tv.transform(X_test)

In [ ]:
cv_train_reviews

<1280000x8128882 sparse matrix of type '<class 'numpy.int64'>'
	with 8128882 stored elements in Compressed Sparse Row format>

### Reference Data

Train Data | Test Data
---| ---
Count Vectorizer Data
cv_train_reviews | y_train
cv_test_reviews | y_test
TFIDF Data | 
tv_train_reviews | y_train
tv_test_reviews | y_test

AUTOML 

- PyCaret
- H20 AutoML

PyCaret is an open source, low-code machine learning library in Python that allows you to go from preparing your data to deploying your model within seconds in your choice of notebook environment.

PyCaret being a low-code library makes you more productive. With less time spent coding, you and your team can now focus on business problems.

PyCaret is simple and easy to use machine learning library that will help you to perform end-to-end ML experiments with less lines of code.

PyCaret is a business ready solution. It allows you to do prototyping quickly and efficiently from your choice of notebook environment.

In [ ]:
!pip install h2o

NameError: ignored

In [ ]:
from h2o.automl import H2OAutoML
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp29g86qo8
  JVM stdout: /tmp/tmp29g86qo8/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp29g86qo8/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.3
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_unknownUser_i21on7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.379 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [ ]:
x = h2o.H2OFrame(cv_train_reviews)

Parse progress: |████████████████████████ (failed)


OSError: ignored

In [ ]:
aml = H2OAutoML(nfolds=5,balance_classes=False, seed =42)
aml.train(x = cv_train_reviews, y = y_train, training_frame=None, validation_frame=None)

ValueError: ignored

In [ ]:
import logging
# Logging configuration
logger = logging.getLogger(__name__)
# Level set to Debug
logger.setLevel(logging.DEBUG)
# Utilizing os module to find the current path and append the log name for the file name
filename = os.path.join('/content/FundData.log')
# Mode & File Name defined
fhandler = logging.FileHandler(filename=filename, mode='a')
# Logging format for both log and stream handler
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
# Stream Handler
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
# Log Handlers
logger.addHandler(fhandler)
logger.addHandler(stream_handler)

In [ ]:
Model_Name = 'XGBoost'
a_s = 98.8
precision = 89
recall = 81
logger.debug("Model Name {}".format(Model_Name))
logger.debug("Accuracy Score {}".format(a_s))
logger.debug("Precision Score {}".format(precision))
logger.debug("Recall Score {}".format(recall))

2021-01-24 19:23:40,497 - __main__ - DEBUG - Model Name XGBoost
2021-01-24 19:23:40,500 - __main__ - DEBUG - Accuracy Score 98.8
2021-01-24 19:23:40,502 - __main__ - DEBUG - Precision Score 89
2021-01-24 19:23:40,503 - __main__ - DEBUG - Recall Score 81
